In [1]:
import sys, torch
print("Python:", sys.executable)
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
from dataclasses import dataclass
from typing import Any, Dict, List
from datasets import load_dataset
import torch
from transformers import (
    AutoProcessor, MllamaForConditionalGeneration,
    BitsAndBytesConfig, TrainingArguments
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
from PIL import Image
import pandas as pd
from pathlib import Path
import json, random
import argparse
from datasets import load_dataset
from transformers import AutoProcessor, MllamaForConditionalGeneration, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
from PIL import Image
import torch, os, re
from dataclasses import dataclass
import pandas as pd

Python: /home/hice1/wyiu31/scratch/venvs/llama32v/bin/python
Torch: 2.5.1+cu121
CUDA available: True
GPU: NVIDIA H100 80GB HBM3


/home/hice1/wyiu31/scratch/venvs/llama32v/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
from transformers import MllamaForConditionalGeneration, AutoProcessor
from PIL import Image
import torch

# Use your local model copy
LOCAL_DIR = "/home/hice1/wyiu31/scratch/models/llama32v"
os.environ["TRANSFORMERS_OFFLINE"] = "1"

# Safer default on shared GPUs (auto offload if VRAM is tight)
model = MllamaForConditionalGeneration.from_pretrained(
    LOCAL_DIR, device_map="auto", dtype=torch.float16, local_files_only=True
)
processor = AutoProcessor.from_pretrained(LOCAL_DIR, local_files_only=True)
print("Model & processor loaded")

Loading checkpoint shards: 100%|██████████| 5/5 [00:07<00:00,  1.50s/it]


Model & processor loaded


In [3]:
#!/usr/bin/env python3
# make_jsonl_with_images.py
from __future__ import annotations

import json, re
from pathlib import Path
from datetime import datetime
from typing import List, Optional

import pandas as pd

# ========= Paths (edit if needed) =========
CSV_PATH = Path("/home/hice1/wyiu31/scratch/stonemt_cameratrap/Camera Trap Photos/Observations_2022.csv")
IMG_BASE = Path("/home/hice1/wyiu31/scratch/stonemt_cameratrap/Camera Trap Photos/output")
OUT_JSONL = Path("/home/hice1/wyiu31/scratch/stonemt_cameratrap/Camera Trap Photos/label_json/Observations_2022_with_images_v2.jsonl")

# ========= Helpers =========

EXT_RE = re.compile(r"\.(jpg|jpeg|png|bmp|gif|webp)$", re.IGNORECASE)

def parse_date_mmddyyyy(s: str) -> Optional[datetime]:
    """CSV has dates like 03/01/2022. Return datetime or None."""
    if not isinstance(s, str):
        return None
    s = s.strip()
    if not s:
        return None
    for fmt in ("%m/%d/%Y", "%m/%d/%y"):
        try:
            return datetime.strptime(s, fmt)
        except Exception:
            pass
    return None

def to_mm_dd_yyyy(d: datetime) -> str:
    """Folder format: MM-DD-YYYY."""
    return d.strftime("%m-%d-%Y")

def norm_common_name(s: str) -> str:
    """Keep as-is but trimmed; no 'Common Name:' prefix."""
    return str(s).strip()

def norm_location(s: str) -> str:
    """Location folder is used as-is, but trim whitespace."""
    return str(s).strip()

def norm_imagename(s: str) -> str:
    """
    We search by 'starts with ImageName + "__"'. Keep the base name w/o extension.
    e.g., 'IMG_0190'  -> match files like 'IMG_0190__20220402_184851__abc.JPG'
    """
    s = str(s).strip()
    s = re.sub(r"\.(jpg|jpeg|png|bmp|gif|webp)$", "", s, flags=re.IGNORECASE)
    return s

def find_images_for(loc: str, date_folder: str, stem: str) -> List[str]:
    """Look under IMG_BASE/loc/date_folder for files starting with stem + '__'."""
    hits: List[str] = []
    loc_dir = IMG_BASE / loc
    date_dir = loc_dir / date_folder
    if date_dir.exists() and date_dir.is_dir():
        prefix = (stem + "__").upper()
        for p in date_dir.iterdir():
            if p.is_file() and EXT_RE.search(p.name) and p.name.upper().startswith(prefix):
                hits.append(str(p))
    return hits

# ========= Load CSV =========

CSV_PATH.parent.mkdir(parents=True, exist_ok=True)
OUT_JSONL.parent.mkdir(parents=True, exist_ok=True)

df = pd.read_csv(CSV_PATH, sep=None, engine="python", dtype=str)
df.columns = df.columns.str.strip()
# Drop unnamed auto-index cols
df = df.loc[:, ~df.columns.str.match(r"^Unnamed", na=False)]

# Ensure required columns exist
required = {"Location", "ImageName", "CommonName"}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"CSV is missing required columns: {sorted(missing)}")

# Pick date: prefer TrueDate, then CameraDate
true_dates = df["TrueDate"] if "TrueDate" in df.columns else None
camera_dates = df["CameraDate"] if "CameraDate" in df.columns else None

picked_dates: list[Optional[datetime]] = []
for i in range(len(df)):
    d = None
    if true_dates is not None:
        d = parse_date_mmddyyyy(true_dates.iloc[i])
    if d is None and camera_dates is not None:
        d = parse_date_mmddyyyy(camera_dates.iloc[i])
    picked_dates.append(d)

df["_picked_date"] = picked_dates
df = df.dropna(subset=["Location", "ImageName", "CommonName"], how="any")

# ========= Build JSONL =========

total = 0
wrote = 0
no_date = 0
no_img  = 0

unmatched_samples = []  # for debugging printout

with OUT_JSONL.open("w", encoding="utf-8") as fout:
    for _, row in df.iterrows():
        total += 1
        loc = norm_location(row["Location"])
        stem = norm_imagename(row["ImageName"])
        label = norm_common_name(row["CommonName"])
        dt: Optional[datetime] = row["_picked_date"]

        if dt is None:
            no_date += 1
            # skip rows with no usable date (we don't know the folder)
            continue

        date_folder = to_mm_dd_yyyy(dt)
        imgs = find_images_for(loc, date_folder, stem)

        if not imgs:
            no_img += 1
            if len(unmatched_samples) < 10:
                unmatched_samples.append({
                    "Location": loc,
                    "date_folder": date_folder,
                    "stem": stem,
                    "hint_dir": str(IMG_BASE / loc / date_folder)
                })
            # skip rows with no matching image
            continue

        rec = {
            "metadata": {
                "location": loc,
                "date": date_folder,  # keep folder-ready format
                "stem": stem,
            },
            "images": imgs,            # list[str]
            "output": label,           # plain common name
        }
        fout.write(json.dumps(rec, ensure_ascii=False) + "\n")
        wrote += 1

print(f"\nDone.")
print(f"Read rows             : {len(df):,}")
print(f"Considered total      : {total:,}")
print(f"Wrote JSONL entries   : {wrote:,}")
print(f"Skipped (no date)     : {no_date:,}")
print(f"Skipped (no image hit): {no_img:,}")
if unmatched_samples:
    print("\nExamples with no image match (up to 10):")
    for ex in unmatched_samples:
        print(f"  - loc={ex['Location']}  date={ex['date_folder']}  stem={ex['stem']}  dir={ex['hint_dir']}")
print(f"\nSaved to: {OUT_JSONL}")


Done.
Read rows             : 11,608
Considered total      : 11,608
Wrote JSONL entries   : 6,273
Skipped (no date)     : 0
Skipped (no image hit): 5,335

Examples with no image match (up to 10):
  - loc=SM_4  date=02-11-2022  stem=IMG_0111  dir=/home/hice1/wyiu31/scratch/stonemt_cameratrap/Camera Trap Photos/output/SM_4/02-11-2022
  - loc=SM_4  date=02-11-2022  stem=IMG_0112  dir=/home/hice1/wyiu31/scratch/stonemt_cameratrap/Camera Trap Photos/output/SM_4/02-11-2022
  - loc=SM_4  date=02-11-2022  stem=IMG_0113  dir=/home/hice1/wyiu31/scratch/stonemt_cameratrap/Camera Trap Photos/output/SM_4/02-11-2022
  - loc=SM_4  date=02-11-2022  stem=IMG_0114  dir=/home/hice1/wyiu31/scratch/stonemt_cameratrap/Camera Trap Photos/output/SM_4/02-11-2022
  - loc=SM_4  date=02-11-2022  stem=IMG_0116  dir=/home/hice1/wyiu31/scratch/stonemt_cameratrap/Camera Trap Photos/output/SM_4/02-11-2022
  - loc=SM_4  date=02-11-2022  stem=IMG_0117  dir=/home/hice1/wyiu31/scratch/stonemt_cameratrap/Camera Trap Photo

In [4]:
import json, re
from pathlib import Path

# Path to your observation file
path = Path("/home/hice1/wyiu31/scratch/stonemt_cameratrap/Camera Trap Photos/label_json/Observations_2022_with_images_v2.jsonl")

total, unknown = 0, 0

with path.open() as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        total += 1
        try:
            rec = json.loads(line)
            # Combine possible text fields (adjust these keys if your schema differs)
            text = json.dumps(rec)
            if re.search(r"\bUnknown\b", text, re.IGNORECASE):
                unknown += 1
        except json.JSONDecodeError:
            continue

print(f"Lines processed: {total}")
if total > 0:
    print(f"Unknown labels: {unknown}/{total} = {unknown/total:.1%}")
else:
    print("No valid lines found.")

Lines processed: 6273
Unknown labels: 0/6273 = 0.0%
